In [2]:
pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163340 sha256=ec77fa506e14f70e401e8e499cd98a78ff1d0c61837188b1abcea5e858c9fca2
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [3]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
import random

In [4]:
# Load dataset
df = pd.read_csv('/content/drive/MyDrive/Virtual Agriculture Marketplace/AgroPlace_Dataset_6.csv')

In [5]:
# Create a Reader object to specify the rating scale
reader = Reader(rating_scale=(0, 10))

data = Dataset.load_from_df(df[['Customer ID', 'Product line', 'Rating']], reader)

# Split the dataset into training and testing sets
trainset = data.build_full_trainset()

# Build and train the item-based collaborative filtering model
sim_options = {
    'name': 'cosine',
    'user_based': False  # Item-based collaborative filtering
}

model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [6]:
# Get a list of unique user IDs
user_ids = trainset.all_users()

# Randomly select a user ID
random_user_id = random.choice(user_ids)

# Retrieve a list of all product IDs
product_ids = list(df['Product line'].unique())
random.shuffle(product_ids)

In [7]:
# Get predictions
predictions = [model.predict(random_user_id, trainset.to_inner_iid(item_id)) for item_id in product_ids]

# Sort the predictions by estimated rating
sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

# Get the top 5 recommended item
top_n_recommendations = sorted_predictions[:5]

# Create a dictionary to map inner item IDs to product line names
id_to_product_name = {trainset.to_inner_iid(item_id): item_id for item_id in product_ids}

# Convert inner item IDs to product line names
recommended_items = [id_to_product_name[prediction.iid] for prediction in top_n_recommendations]

In [8]:
# Print the top recommended items for the random user
print(f"Top 5 recommendations for User {random_user_id}:")
for i, item in enumerate(recommended_items, start=1):
    print(f"{i}. {item}")

Top 5 recommendations for User 2051:
1. Peppers
2. Spinach
3. Onions
4. Wheat
5. Potatoes
